## EDA 결과 요약
- raw data의 31개 columns 중 record의 geo관련 정보가 포함된 12개의 column에 대한 조사
- 각 도시 및 통계 항목 별로 통일 되어있는지 조사 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt 
import missingno as msno
import seaborn as sns 
import os

In [2]:
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("fivethirtyeight")
# 이미지가 레이아웃 안으로 들어오도록 함
mpl.rcParams.update({"figure.constrained_layout.use": True})

In [3]:
import matplotlib.font_manager as fm
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
[fm.FontProperties(fname=font).get_name() for font in font_list if 'D2C' in font]
plt.rc('font', family='D2Coding')
mpl.rcParams['axes.unicode_minus'] = False

In [4]:
DATASET_DIR = '/home/doeun/code/AI/ESTSOFT2024/workspace/dataset/'
dir_name = 'america_big_cities_health_inventory'
file_name = 'BigCitiesHealth.csv'

In [5]:
RSLT_DIR = '/home/doeun/code/AI/ESTSOFT2024/workspace/1.project1_structured/BCHI/processed/'

In [6]:
file_path = os.path.join(DATASET_DIR,dir_name,file_name)
df = pd.read_csv(file_path)

df.info()

/tmp/ipykernel_296727/664991473.py:2: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189979 entries, 0 to 189978
Data columns (total 31 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   metric_item_label                189979 non-null  object 
 1   metric_cat_label                 189979 non-null  object 
 2   metric_subcat_label              189979 non-null  object 
 3   metric_item_label_subtitle       189979 non-null  object 
 4   metric_cat_item_yaxis_label      189979 non-null  object 
 5   metric_source_desc_label_fn      189979 non-null  object 
 6   metric_source_desc_label_url_fn  189979 non-null  object 
 7   geo_label_city                   189979 non-null  object 
 8   geo_label_state                  188667 non-null  object 
 9   geo_label_citystate              189979 non-null  object 
 10  geo_fips_code                    188667 non-null  float64
 11  value                            189979 non-null  float64
 12  da

In [7]:
df["metric_label"] = df["metric_subcat_label"]+' | '+df["metric_item_label"] 

In [8]:
df["metric_label"].value_counts()

metric_label
Deaths | Premature Death                                  6825
Deaths | Injury Deaths                                    6405
Deaths | Deaths from All Causes                           6244
Cancer | All Cancer Deaths                                6233
Cardiovascular Disease | Cardiovascular Disease Deaths    6195
                                                          ... 
Housing | Homeless, Total                                  266
Housing | Vacant Housing and Homelessness                  266
Housing | Homeless, Children                               256
Respiratory Infection | COVID-19 Vaccination, Adults        70
Respiratory Infection | COVID-19 Vaccination, Seniors       70
Name: count, Length: 118, dtype: int64

In [9]:
df.columns

Index(['metric_item_label', 'metric_cat_label', 'metric_subcat_label',
       'metric_item_label_subtitle', 'metric_cat_item_yaxis_label',
       'metric_source_desc_label_fn', 'metric_source_desc_label_url_fn',
       'geo_label_city', 'geo_label_state', 'geo_label_citystate',
       'geo_fips_code', 'value', 'date_label', 'geo_label_proxy_or_real',
       'geo_label_proxy_footnote', 'geo_fips_desc', 'date_label_proxy_or_real',
       'date_label_proxy_footnote', 'value_ci_flag_yesno', 'value_95_ci_low',
       'value_95_ci_high', 'value_90_ci_low', 'value_90_ci_high',
       'geo_strata_region', 'geo_strata_poverty', 'geo_strata_Population',
       'geo_strata_PopDensity', 'geo_strata_Segregation', 'strata_race_label',
       'strata_sex_label', 'strata_race_sex_label', 'metric_label'],
      dtype='object')

1. geo들어간 데이터 체크

In [10]:
cond = df['geo_label_city'] == 'U.S. Total'

df_US = df[cond]
df_city = df[~cond]

display(len(df_US))
display(len(df_city))

1312

188667

In [11]:
list(filter(lambda x: 'geo' in x,df.columns))

['geo_label_city',
 'geo_label_state',
 'geo_label_citystate',
 'geo_fips_code',
 'geo_label_proxy_or_real',
 'geo_label_proxy_footnote',
 'geo_fips_desc',
 'geo_strata_region',
 'geo_strata_poverty',
 'geo_strata_Population',
 'geo_strata_PopDensity',
 'geo_strata_Segregation']

## geo_proxy

In [12]:
display(df_city.geo_label_proxy_footnote.value_counts())
display(df_city.geo_label_proxy_or_real.value_counts())
display(df_US.geo_label_proxy_footnote.value_counts())
display(df_US.geo_label_proxy_or_real.value_counts())

geo_label_proxy_footnote
Data are for school districts    9867
County level data were used      6655
Name: count, dtype: int64

geo_label_proxy_or_real
real     162663
proxy     26004
Name: count, dtype: int64

geo_label_proxy_footnote
Data are for school districts    198
Average for BCHC cities           52
County level data were used       20
Average of all BCHC cities        12
Name: count, dtype: int64

geo_label_proxy_or_real
real     878
proxy    434
Name: count, dtype: int64

In [13]:
display(np.sum(df_city.geo_label_proxy_footnote.isna()))
display(np.sum(df_city.geo_label_proxy_or_real.isna()))
display(np.sum(df_US.geo_label_proxy_footnote.isna()))
display(np.sum(df_US.geo_label_proxy_or_real.isna()))

172145

0

1030

0

In [14]:
cond = df_city.geo_label_proxy_footnote.isna()

display(df_city.loc[cond,'geo_label_proxy_or_real'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_or_real'].isna()))

geo_label_proxy_or_real
real     162663
proxy      9482
Name: count, dtype: int64

0

In [15]:
df_geoproxy_counts =df_city[['metric_label','geo_label_proxy_or_real','geo_label_proxy_footnote']].value_counts().reset_index()
df_geoproxy_counts

,metric_label,geo_label_proxy_or_real,geo_label_proxy_footnote,count
0,Substance Use | Teen Marijuana,proxy,Data are for school districts,563
1,Diabetes and Obesity | Teen Obesity,proxy,Data are for school districts,560
2,Substance Use | Teen Alcohol,proxy,Data are for school districts,557
3,Physical Activity | Teen Physical Activity,proxy,Data are for school districts,555
4,Mental Health | School Bullying,proxy,Data are for school districts,548
5,Mental Health | Electronic Bullying,proxy,Data are for school districts,547
6,Mental Health | Teen Mental Distress,proxy,Data are for school districts,547
7,Substance Use | Teen Smoking,proxy,Data are for school districts,545
8,Physical Activity | Teen TV Time,proxy,Data are for school districts,544
9,Physical Activity | Teen Physical Inactivity,proxy,Data are for school districts,538


In [16]:
np.sum(df_geoproxy_counts.duplicated(subset=['metric_label'],keep=False))

0

In [17]:
df_temp = df_city.copy()
df_temp['geo_label_proxy_footnote'] = df_city['geo_label_proxy_footnote'].fillna('NAN')
df_geoproxy_counts2 =df_temp[['metric_label','geo_label_proxy_or_real','geo_label_proxy_footnote']].value_counts().reset_index()

np.sum(df_geoproxy_counts2.duplicated(subset=['metric_label'],keep=False))

54

In [18]:
df_geoproxy_counts2['geo_label_proxy_or_real'].value_counts()

geo_label_proxy_or_real
real     86
proxy    59
Name: count, dtype: int64

In [19]:
cond_dup = (df_geoproxy_counts2.duplicated(subset=['metric_label'],keep=False))
df_geoproxy_counts2[cond_dup].sort_values(by=['metric_label','geo_label_proxy_or_real'])

,metric_label,geo_label_proxy_or_real,geo_label_proxy_footnote,count
134,Births | Teen Births,proxy,County level data were used,129
34,Births | Teen Births,real,NAN,2106
117,Cancer | All Cancer Deaths,proxy,County level data were used,312
3,Cancer | All Cancer Deaths,real,NAN,5908
136,Cancer | Breast Cancer Deaths,proxy,County level data were used,104
38,Cancer | Breast Cancer Deaths,real,NAN,1669
124,Cancer | Colorectal Cancer Deaths,proxy,County level data were used,304
11,Cancer | Colorectal Cancer Deaths,real,NAN,4740
126,Cancer | Lung Cancer Deaths,proxy,County level data were used,296
6,Cancer | Lung Cancer Deaths,real,NAN,5179


In [20]:
cond = df_geoproxy_counts2['geo_label_proxy_footnote'] == 'NAN'
df_geoproxy_counts2.loc[cond,'geo_label_proxy_or_real'].value_counts()

geo_label_proxy_or_real
real     86
proxy    13
Name: count, dtype: int64

In [21]:
cond0 = df_geoproxy_counts2['geo_label_proxy_footnote'] == 'NAN'
cond1 = df_geoproxy_counts2['geo_label_proxy_or_real'] == 'proxy'
df_geoproxy_counts2.loc[cond0 & cond1]

,metric_label,geo_label_proxy_or_real,geo_label_proxy_footnote,count
18,Sexually Transmitted Disease | HIV/AIDS Preval...,proxy,NAN,2897
37,"Respiratory Infection | Flu Vaccinations, Medi...",proxy,NAN,1995
65,Respiratory Infection | New Tuberculosis Cases,proxy,NAN,455
70,"Sexually Transmitted Disease | Syphilis, Newborns",proxy,NAN,455
86,Heat and Disasters | Climate-related Disasters,proxy,NAN,455
96,Air Pollution | Hazardous Air Quality,proxy,NAN,454
97,Air Pollution | Poor Air Quality,proxy,NAN,454
98,Sexually Transmitted Disease | Syphilis Preval...,proxy,NAN,452
99,Sexually Transmitted Disease | New Gonorrhea C...,proxy,NAN,452
100,Sexually Transmitted Disease | New Chlamydia C...,proxy,NAN,452


- geo_proxy_footnote가 na인 경우를 제외하면
    - metric_label 별로, geo_proxy 들은 통일 되어있는 것을 확인 
- geo_proxy_footnote가 na가 있는 경우를 보면
    - 대부분은 통일 되어있음
    - 통일 되어있지 않은 케이스
        - child lead level을 제외하고는 footnote가 NAN인 경우 누락으로 추측 가능
        - 누락된 경우 proxy_or_real이 real로 채워졌다 추측 가능

In [22]:
cond = df_city.geo_label_proxy_or_real == 'proxy'

display(df_city.loc[cond,'geo_label_proxy_footnote'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_footnote'].isna()))

geo_label_proxy_footnote
Data are for school districts    9867
County level data were used      6655
Name: count, dtype: int64

9482

- school districts

In [23]:
cond0 = df_city.geo_label_proxy_or_real == 'proxy'
cond1 = df_city.geo_label_proxy_footnote == 'Data are for school districts' 
display(df_city.loc[cond0 & cond1,'metric_label'].value_counts())
display(df_city.loc[cond0 & cond1,'geo_label_city'].value_counts())
display(len(df_city.loc[cond0 & cond1,'geo_label_city'].value_counts()))

metric_label
Substance Use | Teen Marijuana                  563
Diabetes and Obesity | Teen Obesity             560
Substance Use | Teen Alcohol                    557
Physical Activity | Teen Physical Activity      555
Mental Health | School Bullying                 548
Mental Health | Teen Mental Distress            547
Mental Health | Electronic Bullying             547
Substance Use | Teen Smoking                    545
Physical Activity | Teen TV Time                544
Physical Activity | Teen Physical Inactivity    538
School Violence | Fighting in School            520
Physical Activity | Teen Computer Time          518
School Violence | Weapons in School             509
Dietary Quality | Teen Soda                     507
Respiratory Disease | Teen Asthma               506
Mental Health | Teen Suicidal Ideation          497
Dietary Quality | Teen Breakfast                475
Physical Activity | Teen Physical Education     444
Birth Control | Teen Birth Control              387

geo_label_city
Philadelphia     756
Boston           737
New York City    719
Houston          702
San Diego        680
San Francisco    635
Los Angeles      626
Washington       601
Chicago          555
Detroit          508
Fort Worth       488
Oakland          472
Seattle          472
Cleveland        406
Baltimore        386
Charlotte        386
Portland         318
Dallas           269
Las Vegas        151
Name: count, dtype: int64

19

In [24]:
label_proxy_school = set(df_city.loc[cond0 & cond1,'metric_label'].unique())
display(label_proxy_school)
display(len(label_proxy_school))
cond = df_city['metric_label'].isin(label_proxy_school)
display(df_city.loc[cond,'geo_label_proxy_or_real'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_or_real'].isna()))
display(df_city.loc[cond,'geo_label_proxy_footnote'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_footnote'].isna()))

{'Birth Control | Teen Birth Control',
 'Diabetes and Obesity | Teen Obesity',
 'Dietary Quality | Teen Breakfast',
 'Dietary Quality | Teen Soda',
 'Mental Health | Electronic Bullying',
 'Mental Health | School Bullying',
 'Mental Health | Teen Mental Distress',
 'Mental Health | Teen Suicidal Ideation',
 'Physical Activity | Teen Computer Time',
 'Physical Activity | Teen Physical Activity',
 'Physical Activity | Teen Physical Education',
 'Physical Activity | Teen Physical Inactivity',
 'Physical Activity | Teen TV Time',
 'Respiratory Disease | Teen Asthma',
 'School Violence | Fighting in School',
 'School Violence | Weapons in School',
 'Substance Use | Teen Alcohol',
 'Substance Use | Teen Marijuana',
 'Substance Use | Teen Smoking'}

19

geo_label_proxy_or_real
proxy    9867
Name: count, dtype: int64

0

geo_label_proxy_footnote
Data are for school districts    9867
Name: count, dtype: int64

0

In [25]:
label_school = list(filter(lambda x : (('school' in x.lower()) or ('teen' in x.lower())),
                           df_city['metric_label'].unique())) 
cond = df_city['metric_label'].isin(label_school)
display(len(label_school))
display(df_city.loc[cond,'geo_label_proxy_or_real'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_or_real'].isna()))
display(df_city.loc[cond,'geo_label_proxy_footnote'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_footnote'].isna()))

20

geo_label_proxy_or_real
proxy    9449
real     2561
Name: count, dtype: int64

0

geo_label_proxy_footnote
Data are for school districts    9320
County level data were used       129
Name: count, dtype: int64

2561

In [26]:
label_school = list(filter(lambda x : (('school' in x.lower()) or ('teen' in x.lower())) or ('bullying' in x.lower()),
                           df_city['metric_label'].unique())) 
cond = df_city['metric_label'].isin(label_school)
display(len(label_school))
display(df_city.loc[cond,'geo_label_proxy_or_real'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_or_real'].isna()))
display(df_city.loc[cond,'geo_label_proxy_footnote'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_footnote'].isna()))
display(len(df_city.loc[cond,'geo_label_city'].value_counts()))

21

geo_label_proxy_or_real
proxy    9996
real     2561
Name: count, dtype: int64

0

geo_label_proxy_footnote
Data are for school districts    9867
County level data were used       129
Name: count, dtype: int64

2561

35

In [27]:
label_teen = list(filter(lambda x : ('teen' in x.lower()),
                           df_city['metric_label'].unique()))
display(label_teen)
cond = df_city['metric_label'].isin(label_teen)
display(df_city.loc[cond,'geo_label_proxy_or_real'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_or_real'].isna()))
display(df_city.loc[cond,'geo_label_proxy_footnote'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_footnote'].isna()))
display(len(df_city.loc[cond,'geo_label_city'].value_counts()))

['Diabetes and Obesity | Teen Obesity',
 'Respiratory Disease | Teen Asthma',
 'Physical Activity | Teen Physical Activity',
 'Dietary Quality | Teen Soda',
 'Dietary Quality | Teen Breakfast',
 'Physical Activity | Teen Computer Time',
 'Physical Activity | Teen TV Time',
 'Physical Activity | Teen Physical Inactivity',
 'Physical Activity | Teen Physical Education',
 'Births | Teen Births',
 'Birth Control | Teen Birth Control',
 'Substance Use | Teen Smoking',
 'Mental Health | Teen Mental Distress',
 'Mental Health | Teen Suicidal Ideation',
 'Substance Use | Teen Alcohol',
 'Substance Use | Teen Marijuana']

geo_label_proxy_or_real
proxy    7872
real     2106
Name: count, dtype: int64

0

geo_label_proxy_footnote
Data are for school districts    7743
County level data were used       129
Name: count, dtype: int64

2106

35

In [28]:
label_notprxy_teen = list(filter(lambda x : x not in label_proxy_school, label_teen))
display(label_notprxy_teen)
cond = df_city['metric_label'].isin(label_notprxy_teen)
display(df_city.loc[cond,'geo_label_proxy_or_real'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_or_real'].isna()))
display(df_city.loc[cond,'geo_label_proxy_footnote'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_footnote'].isna()))

['Births | Teen Births']

geo_label_proxy_or_real
real     2106
proxy     129
Name: count, dtype: int64

0

geo_label_proxy_footnote
County level data were used    129
Name: count, dtype: int64

2106

- county level

In [29]:
cond0 = df_city.geo_label_proxy_or_real == 'proxy'
cond1 = df_city.geo_label_proxy_footnote == 'County level data were used'
display(df_city.loc[cond0 & cond1,'metric_label'].value_counts())
display(df_city.loc[cond0 & cond1,'geo_label_city'].value_counts())

metric_label
Deaths | Premature Death                                  390
Deaths | Injury Deaths                                    370
Substance Use | Drug Overdose Deaths                      326
Mental Health | Suicide                                   315
Deaths | Gun Deaths (Firearms)                            315
Cancer | All Cancer Deaths                                312
Cardiovascular Disease | Cardiovascular Disease Deaths    311
Cardiovascular Disease | Heart Disease Deaths             311
Crime Incidents | Homicides                               310
Deaths | Motor Vehicle Deaths                             308
Deaths | Deaths from All Causes                           308
Substance Use | Opioid Overdose Deaths                    305
Cancer | Colorectal Cancer Deaths                         304
Diabetes and Obesity | Diabetes Deaths                    299
Cancer | Lung Cancer Deaths                               296
Respiratory Infection | Pneumonia or Influenza Deaths    

geo_label_city
Las Vegas        3617
Louisville       2450
Columbus           28
Los Angeles        28
Memphis            28
Oakland            28
Portland           28
Phoenix            28
San Jose           28
El Paso            28
Milwaukee          28
Cleveland          28
Charlotte          28
San Diego          28
Fort Worth         28
Austin             28
Minneapolis        28
Seattle            28
San Antonio        28
Tucson             24
Denver             16
Dallas             12
Houston            12
New York City       4
San Francisco       4
Philadelphia        4
Long Beach          4
Oklahoma City       4
Boston              4
Kansas City         4
Indianapolis        4
Chicago             4
Washington          4
Detroit             4
Baltimore           4
Name: count, dtype: int64

In [30]:
label_proxy_county = set(df_city.loc[cond0 & cond1,'metric_label'].unique())
cond = df_city['metric_label'].isin(label_proxy_county)
display(df_city.loc[cond,'geo_label_proxy_or_real'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_or_real'].isna()))
display(df_city.loc[cond,'geo_label_proxy_footnote'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_footnote'].isna()))

geo_label_proxy_or_real
real     98684
proxy     6655
Name: count, dtype: int64

0

geo_label_proxy_footnote
County level data were used    6655
Name: count, dtype: int64

98684

In [31]:
cond0 = df_city.geo_label_proxy_or_real == 'proxy'
cond1 = df_city.geo_label_proxy_footnote.isna()
display(df_city.loc[cond0 & cond1,'metric_label'].value_counts())
display(df_city.loc[cond0 & cond1,'geo_label_city'].value_counts())

metric_label
Sexually Transmitted Disease | HIV/AIDS Prevalence    2897
Respiratory Infection | Flu Vaccinations, Medicare    1995
Respiratory Infection | New Tuberculosis Cases         455
Sexually Transmitted Disease | Syphilis, Newborns      455
Heat and Disasters | Climate-related  Disasters        455
Air Pollution | Poor Air Quality                       454
Air Pollution | Hazardous Air Quality                  454
Sexually Transmitted Disease | New Chlamydia Cases     452
Sexually Transmitted Disease | Syphilis Prevalence     452
Sexually Transmitted Disease | New Gonorrhea Cases     452
Lead Poisoning | Child Lead Testing                    350
Lead Poisoning | Child Lead Levels 5+ mcg/dL           344
Lead Poisoning | Child Lead Levels 10+ mcg/dL          267
Name: count, dtype: int64

geo_label_city
New York City    287
Phoenix          287
Detroit          287
Minneapolis      287
Seattle          287
Indianapolis     283
Philadelphia     283
Denver           279
Kansas City      278
Oakland          278
Columbus         278
Cleveland        278
San Diego        278
Memphis          278
Charlotte        278
Los Angeles      278
Milwaukee        277
Washington       273
Tucson           272
Oklahoma City    271
Dallas           270
El Paso          269
San Jose         268
Chicago          268
Fort Worth       267
Houston          267
Austin           267
San Antonio      267
San Francisco    267
Baltimore        258
Portland         254
Las Vegas        254
Louisville       246
Long Beach       245
Boston           218
Name: count, dtype: int64

In [32]:
label_proxy_na = set(df_city.loc[cond0 & cond1,'metric_label'].unique())
cond = df_city['metric_label'].isin(label_proxy_na)
display(df_city.loc[cond,'geo_label_proxy_or_real'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_or_real'].isna()))
display(df_city.loc[cond,'geo_label_proxy_footnote'].value_counts())
display(np.sum(df_city.loc[cond,'geo_label_proxy_footnote'].isna()))

geo_label_proxy_or_real
proxy    9482
real       22
Name: count, dtype: int64

0

Series([], Name: count, dtype: int64)

9504

- geo_label_proxy_or_real이 proxy인 metric label의 경우 바깥의 데이터를 모델에 이식할 때 ratio가 아닌 경우 특히 주의 필요

## geo_label

In [33]:
list(filter(lambda x: 'geo' in x,df.columns))

['geo_label_city',
 'geo_label_state',
 'geo_label_citystate',
 'geo_fips_code',
 'geo_label_proxy_or_real',
 'geo_label_proxy_footnote',
 'geo_fips_desc',
 'geo_strata_region',
 'geo_strata_poverty',
 'geo_strata_Population',
 'geo_strata_PopDensity',
 'geo_strata_Segregation']

In [34]:
display(np.sum(df_city['geo_label_city'].isna()))
display(np.sum(df_city['geo_label_state'].isna()))
display(np.sum(df_city['geo_label_citystate'].isna()))

0

0

0

In [35]:
df_city_label =df_city[['geo_label_city','geo_label_state','geo_label_citystate']].value_counts().reset_index()

np.sum(df_city_label.duplicated(subset=['geo_label_city'],keep=False))

0

In [36]:
df_city_label

,geo_label_city,geo_label_state,geo_label_citystate,count
0,New York City,NY,"New York City, NY",6535
1,Houston,TX,"Houston, TX",6431
2,Los Angeles,CA,"Los Angeles, CA",6372
3,Philadelphia,PA,"Philadelphia, PA",6310
4,San Diego,CA,"San Diego, CA",6251
5,Chicago,IL,"Chicago, IL",6210
6,San Francisco,CA,"San Francisco, CA",6121
7,Las Vegas,NV,"Las Vegas, NV",5849
8,Boston,MA,"Boston, MA",5718
9,Oakland,CA,"Oakland, CA",5676


## geo_fips

In [37]:
list(filter(lambda x: 'geo' in x,df.columns))

['geo_label_city',
 'geo_label_state',
 'geo_label_citystate',
 'geo_fips_code',
 'geo_label_proxy_or_real',
 'geo_label_proxy_footnote',
 'geo_fips_desc',
 'geo_strata_region',
 'geo_strata_poverty',
 'geo_strata_Population',
 'geo_strata_PopDensity',
 'geo_strata_Segregation']

In [38]:
display(df_city['geo_fips_desc'].value_counts())
display(np.sum(df_city['geo_fips_desc'].isna()))
display(np.sum(df_city['geo_fips_code'].isna()))

geo_fips_desc
place     178541
county     10126
Name: count, dtype: int64

0

0

In [39]:
df_fips_cnt =df_city[['geo_label_city','geo_fips_code','geo_fips_desc']].value_counts().reset_index()
df_fips_cnt = df_fips_cnt.sort_values(by=['geo_label_city','geo_fips_desc']).reset_index(drop = True)

display(np.sum(df_fips_cnt.duplicated(subset=['geo_label_city'],keep=False)))
display(df_fips_cnt)

70

,geo_label_city,geo_fips_code,geo_fips_desc,count
0,Austin,4805000.0,county,295
1,Austin,4805000.0,place,4903
2,Baltimore,2404000.0,county,262
3,Baltimore,2404000.0,place,4839
4,Boston,2507000.0,county,222
...,...,...,...,...
65,Seattle,5363000.0,place,5192
66,Tucson,477000.0,county,296
67,Tucson,477000.0,place,4503
68,Washington,1150000.0,county,277


In [40]:
df_county_check =df_temp[['geo_label_city','geo_fips_desc','geo_label_proxy_or_real','geo_label_proxy_footnote']].value_counts().reset_index()
df_county_check = df_county_check.sort_values(by=['geo_label_city','geo_fips_desc','geo_label_proxy_or_real']).reset_index(drop = True)

display(np.sum(df_county_check.duplicated(subset=['geo_label_city','geo_fips_desc','geo_label_proxy_or_real'],keep=False)))
display(df_county_check)

72

,geo_label_city,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
0,Austin,county,proxy,NAN,267
1,Austin,county,proxy,County level data were used,28
2,Austin,place,real,NAN,4903
3,Baltimore,county,proxy,NAN,258
4,Baltimore,county,proxy,County level data were used,4
...,...,...,...,...,...
121,Tucson,place,real,NAN,4503
122,Washington,county,proxy,NAN,273
123,Washington,county,proxy,County level data were used,4
124,Washington,place,proxy,Data are for school districts,601


In [41]:
cond = df_county_check['geo_label_city'] == 'Las Vegas'
df_county_check[cond]

,geo_label_city,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
54,Las Vegas,county,proxy,NAN,254
55,Las Vegas,county,proxy,County level data were used,28
56,Las Vegas,place,proxy,County level data were used,3589
57,Las Vegas,place,proxy,Data are for school districts,151
58,Las Vegas,place,real,NAN,1827


In [42]:
cond = df_county_check['geo_label_proxy_footnote'] == 'Data are for school districts'
display(df_county_check[cond])
display(len(df_county_check[cond]))

,geo_label_city,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
5,Baltimore,place,proxy,Data are for school districts,386
9,Boston,place,proxy,Data are for school districts,737
13,Charlotte,place,proxy,Data are for school districts,386
17,Chicago,place,proxy,Data are for school districts,555
21,Cleveland,place,proxy,Data are for school districts,406
28,Dallas,place,proxy,Data are for school districts,269
35,Detroit,place,proxy,Data are for school districts,508
42,Fort Worth,place,proxy,Data are for school districts,488
46,Houston,place,proxy,Data are for school districts,702
57,Las Vegas,place,proxy,Data are for school districts,151


19

- data for school districts에 해당하는 metric label이 없는 도시가 있는지 확인 필요

In [43]:
cond_dup = df_county_check.duplicated(subset=['geo_label_city','geo_fips_desc','geo_label_proxy_or_real'],keep=False)
df_county_temp = df_county_check[cond_dup].sort_values(by=['geo_label_city','geo_fips_desc'])
df_county_temp

,geo_label_city,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
0,Austin,county,proxy,NAN,267
1,Austin,county,proxy,County level data were used,28
3,Baltimore,county,proxy,NAN,258
4,Baltimore,county,proxy,County level data were used,4
7,Boston,county,proxy,NAN,218
...,...,...,...,...,...
116,Seattle,county,proxy,County level data were used,28
119,Tucson,county,proxy,NAN,272
120,Tucson,county,proxy,County level data were used,24
122,Washington,county,proxy,NAN,273


In [44]:
display(df_county_temp['geo_fips_desc'].value_counts())
display(df_county_temp['geo_label_proxy_or_real'].value_counts())
display(df_county_temp['geo_label_proxy_footnote'].value_counts())

geo_fips_desc
county    70
place      2
Name: count, dtype: int64

geo_label_proxy_or_real
proxy    72
Name: count, dtype: int64

geo_label_proxy_footnote
County level data were used      36
NAN                              35
Data are for school districts     1
Name: count, dtype: int64

In [45]:
cond = df_county_temp['geo_label_proxy_footnote'] == 'County level data were used'
df_county_temp.loc[cond,'geo_label_city'].value_counts()

geo_label_city
Las Vegas        2
Austin           1
Louisville       1
Milwaukee        1
Minneapolis      1
New York City    1
Oakland          1
Oklahoma City    1
Philadelphia     1
Phoenix          1
Portland         1
San Antonio      1
San Diego        1
San Francisco    1
San Jose         1
Seattle          1
Tucson           1
Memphis          1
Los Angeles      1
Baltimore        1
Long Beach       1
Kansas City      1
Indianapolis     1
Houston          1
Fort Worth       1
El Paso          1
Detroit          1
Denver           1
Dallas           1
Columbus         1
Cleveland        1
Chicago          1
Charlotte        1
Boston           1
Washington       1
Name: count, dtype: int64

In [46]:
cond = df_county_temp['geo_label_city'] == 'Las Vegas'
df_county_temp[cond]

,geo_label_city,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
54,Las Vegas,county,proxy,NAN,254
55,Las Vegas,county,proxy,County level data were used,28
56,Las Vegas,place,proxy,County level data were used,3589
57,Las Vegas,place,proxy,Data are for school districts,151


- 어떤 metric label인지 몰라도 las vegas에서 해당 metric label에 대해서 feo_fips_desc를 잘못 기입한 것으로 추측
- 나머지의 경우 geo_fips_desc가 county면 county level data가 쓰인 것으로 보면 될 듯

- metric label 별로 geo_fips_desc가 유일한지 확인 필요

In [47]:
df_metric_county =df_temp[['metric_label','geo_label_city','geo_fips_desc','geo_label_proxy_or_real','geo_label_proxy_footnote']].value_counts().reset_index()
df_metric_county = df_metric_county.sort_values(by=['metric_label','geo_label_city','geo_fips_desc','geo_label_proxy_or_real']).reset_index(drop = True)

display(np.sum(df_metric_county.duplicated(subset=['metric_label','geo_fips_desc','geo_label_proxy_or_real'],keep=False)))
display(df_metric_county)

3781

,metric_label,geo_label_city,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
0,Active Transportation | Riding Bike to Work,Austin,place,real,NAN,13
1,Active Transportation | Riding Bike to Work,Baltimore,place,real,NAN,13
2,Active Transportation | Riding Bike to Work,Boston,place,real,NAN,13
3,Active Transportation | Riding Bike to Work,Charlotte,place,real,NAN,13
4,Active Transportation | Riding Bike to Work,Chicago,place,real,NAN,13
...,...,...,...,...,...,...
3778,Transportation | Public Transportation Use,San Francisco,place,real,NAN,63
3779,Transportation | Public Transportation Use,San Jose,place,real,NAN,63
3780,Transportation | Public Transportation Use,Seattle,place,real,NAN,63
3781,Transportation | Public Transportation Use,Tucson,place,real,NAN,63


In [48]:
df_metric_county =df_temp[['metric_label','geo_fips_desc','geo_label_proxy_or_real','geo_label_proxy_footnote']].value_counts().reset_index()
df_metric_county = df_metric_county.sort_values(by=['metric_label','geo_fips_desc','geo_label_proxy_or_real']).reset_index(drop = True)

display(np.sum(df_metric_county.duplicated(subset=['metric_label','geo_fips_desc','geo_label_proxy_or_real'],keep=False)))
display(df_metric_county)

0

,metric_label,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
0,Active Transportation | Riding Bike to Work,place,real,NAN,455
1,Active Transportation | Walking to Work,place,real,NAN,2205
2,Air Pollution | Hazardous Air Quality,county,proxy,NAN,454
3,Air Pollution | Poor Air Quality,county,proxy,NAN,454
4,Birth Control | Teen Birth Control,place,proxy,Data are for school districts,387
...,...,...,...,...,...
140,Substance Use | Teen Smoking,place,proxy,Data are for school districts,545
141,Transportation | Drives Alone to Work,place,real,NAN,2205
142,Transportation | Lack of Car,place,real,NAN,455
143,Transportation | Longer Driving Commute Time,place,real,NAN,350


In [49]:
df_metric_county =df_temp[['metric_label','geo_fips_desc','geo_label_proxy_or_real','geo_label_proxy_footnote']].value_counts().reset_index()
df_metric_county = df_metric_county.sort_values(by=['metric_label','geo_fips_desc']).reset_index(drop = True)

display(np.sum(df_metric_county.duplicated(subset=['metric_label','geo_fips_desc'],keep=False)))
display(df_metric_county)

44

,metric_label,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
0,Active Transportation | Riding Bike to Work,place,real,NAN,455
1,Active Transportation | Walking to Work,place,real,NAN,2205
2,Air Pollution | Hazardous Air Quality,county,proxy,NAN,454
3,Air Pollution | Poor Air Quality,county,proxy,NAN,454
4,Birth Control | Teen Birth Control,place,proxy,Data are for school districts,387
...,...,...,...,...,...
140,Substance Use | Teen Smoking,place,proxy,Data are for school districts,545
141,Transportation | Drives Alone to Work,place,real,NAN,2205
142,Transportation | Lack of Car,place,real,NAN,455
143,Transportation | Longer Driving Commute Time,place,real,NAN,350


In [50]:
cond = (df_metric_county.duplicated(subset=['metric_label','geo_fips_desc'],keep=False))
df_metric_county[cond]

,metric_label,geo_fips_desc,geo_label_proxy_or_real,geo_label_proxy_footnote,count
8,Births | Teen Births,place,real,NAN,2106
9,Births | Teen Births,place,proxy,County level data were used,129
10,Cancer | All Cancer Deaths,place,real,NAN,5908
11,Cancer | All Cancer Deaths,place,proxy,County level data were used,312
12,Cancer | Breast Cancer Deaths,place,real,NAN,1669
13,Cancer | Breast Cancer Deaths,place,proxy,County level data were used,104
14,Cancer | Colorectal Cancer Deaths,place,real,NAN,4740
15,Cancer | Colorectal Cancer Deaths,place,proxy,County level data were used,304
16,Cancer | Lung Cancer Deaths,place,real,NAN,5179
17,Cancer | Lung Cancer Deaths,place,proxy,County level data were used,296


- 결론을 뭔가 내야 함

## geo_strata

In [51]:
col_list = ['geo_strata_region',
 'geo_strata_poverty',
 'geo_strata_Population',
 'geo_strata_PopDensity',
 'geo_strata_Segregation']
for col in col_list:
    display(df_city[col].value_counts())

geo_strata_region
West         66810
South        62748
Midwest      40546
Northeast    18563
Name: count, dtype: int64

geo_strata_poverty
Less poor cities (<18% poor)    141688
Poorest cities (18%+ poor)       46979
Name: count, dtype: int64

geo_strata_Population
Smaller (<1.3 million)    134116
Largest (>1.3 million)     54550
Name: count, dtype: int64

geo_strata_PopDensity
Lower pop. density (<10k per sq mi)      152508
Highest pop. density (>10k per sq mi)     36158
Smaller (<1.3 million)                        1
Name: count, dtype: int64

geo_strata_Segregation
Less Segregated (<50%)      104633
Highly Segregated (50%+)     84034
Name: count, dtype: int64

In [52]:
for col in col_list:
    display(np.sum(df_city[col].isna()))

0

0

1

0

0

In [53]:
df_city[df_city['geo_strata_Population'].isna()]

,metric_item_label,metric_cat_label,metric_subcat_label,metric_item_label_subtitle,metric_cat_item_yaxis_label,metric_source_desc_label_fn,metric_source_desc_label_url_fn,geo_label_city,geo_label_state,geo_label_citystate,...,value_90_ci_high,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,strata_race_label,strata_sex_label,strata_race_sex_label,metric_label
7,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Las Vegas,NV,"Las Vegas, NV",...,NaN,West,Less poor cities (<18% poor),NaN,Lower pop. density (<10k per sq mi),Less Segregated (<50%),All,Male,NaN,Cancer | All Cancer Deaths


In [54]:
cols = ['geo_label_city']+col_list
np.sum(~df_city[cols].duplicated(keep=False))

2

In [55]:
df_city[~df_city[cols].duplicated(keep=False)]

,metric_item_label,metric_cat_label,metric_subcat_label,metric_item_label_subtitle,metric_cat_item_yaxis_label,metric_source_desc_label_fn,metric_source_desc_label_url_fn,geo_label_city,geo_label_state,geo_label_citystate,...,value_90_ci_high,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,strata_race_label,strata_sex_label,strata_race_sex_label,metric_label
3,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Long Beach,CA,"Long Beach, CA",...,NaN,West,Less poor cities (<18% poor),Smaller (<1.3 million),Smaller (<1.3 million),Highly Segregated (50%+),Hispanic,Female,Hispanic Female,Cancer | All Cancer Deaths
7,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Las Vegas,NV,"Las Vegas, NV",...,NaN,West,Less poor cities (<18% poor),NaN,Lower pop. density (<10k per sq mi),Less Segregated (<50%),All,Male,NaN,Cancer | All Cancer Deaths


In [56]:
df_city.loc[df_city['geo_label_city']=='Long Beach',cols].value_counts()

geo_label_city  geo_strata_region  geo_strata_poverty            geo_strata_Population   geo_strata_PopDensity                geo_strata_Segregation  
Long Beach      West               Less poor cities (<18% poor)  Smaller (<1.3 million)  Lower pop. density (<10k per sq mi)  Highly Segregated (50%+)    5215
                                                                                         Smaller (<1.3 million)               Highly Segregated (50%+)       1
Name: count, dtype: int64

In [58]:
df_city[cols].fillna('NAN').value_counts().reset_index(drop=False).sort_values(by =col_list)

,geo_label_city,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,count
5,Chicago,Midwest,Less poor cities (<18% poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),6210
26,Indianapolis,Midwest,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),4916
27,Columbus,Midwest,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),4907
29,Kansas City,Midwest,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),4848
34,Minneapolis,Midwest,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),4511
21,Detroit,Midwest,Poorest cities (18%+ poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),5160
23,Cleveland,Midwest,Poorest cities (18%+ poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),5113
28,Milwaukee,Midwest,Poorest cities (18%+ poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),4881
0,New York City,Northeast,Less poor cities (<18% poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),6535
8,Boston,Northeast,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),5718


**결론**
- las vegas, long beach에 대해서 오기입된 란이 한 record 씩 있음
- 그외의 도시는 연도/record 상관없이 통일되어있음
- 도시간의 metric을 외부에 따로 저장해두고 사용하면 될 듯

## TODO
- 도시간 정보 정리하여 저장하기
- 도시간 metric 만들기
- county 관련 값이 들어간 것들 확실히 체크함 해야할 듯